# Лаба 1. Расчет рейтингов фильмов – RDD

По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

## Описание данных

Имеются следующие входные данные:

* Таблица `users x movies` с рейтингами. Архив с датасетом нужно скачать с сайта [GroupLens](http://files.grouplens.org/datasets/movielens/ml-100k.zip). Также, он загружен на HDFS в `/labs/laba01/ml-100k`. Файл u.data содержит все оценки, а файл u.item — список всех фильмов.

`!hdfs dfs -ls /labs/laba01/ml-100k`

* `id фильма` для расчета индивидуальных характеристик — в Личном кабинете на странице [Лабы 1](https://lk-spark.newprolab.com/lab/slaba01).

## Запуск PySpark

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))


In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "lab01") 

sc = SparkContext(conf=conf)

In [3]:
sc.getConf().getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.app.id', 'application_1613394706435_0552'),
 ('spark.driver.appUIAddress', 'http://spark-de-master-3.newprolab.com:4042'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.history.fs.cleaner.interval', '7d'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://spark-de-master-1.newprolab.com:8088/proxy/application_1613394706435_0552'),
 ('spark.shuffle.io.serverThreads', '128'),
 ('spark.sql.streaming.streamingQueryListeners', ''),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.shuffle.file.buffer', '1m'),
 ('spark.driver.port', '44401'),
 ('spark.sql.hive.convertMetastoreOrc', 'true'),
 ('spark.yarn.dist.files'

Посмотрим на файлы в директории

In [5]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2020-09-05 20:38 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2020-09-05 20:38 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2020-09-05 20:38 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2020-09-05 20:38 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2020-09-05 20:38 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2020-09-05 20:38 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2020-09-05 20:38 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2020-09-05 20:38 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2020-09-05 20:38 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2020-09-05 20:38 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2020-09-05 20:38 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2020-09-05 20:38 /labs/laba01/ml-100k/u2.base


Нам понадобится только таблица с рейтингами

In [4]:
rating_path = '/labs/laba01/ml-100k/u.data'

In [5]:
ratings = sc.textFile(rating_path)

Посмотрим первые 10 записей

In [7]:
ratings.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [8]:
ratings.count()

100000

In [9]:
ratings.getNumPartitions()

2

In [10]:
ratings = sc.textFile("ratings_3_repart.txt")

In [11]:
ratings.getNumPartitions()

3

In [12]:
ratings = ratings.map(lambda x: x.split("\t"))

In [13]:
ratings.take(10)

[['508', '132', '5', '883767279'],
 ['487', '1440', '4', '884045494'],
 ['387', '27', '1', '886483252'],
 ['346', '3', '3', '875265392'],
 ['308', '134', '5', '887737686'],
 ['714', '477', '2', '892777408'],
 ['234', '770', '4', '892335920'],
 ['578', '1264', '3', '890939815'],
 ['624', '591', '3', '879792557'],
 ['655', '124', '3', '887426087']]

Сохраним id фильма в переменную

In [14]:
movie_id = '328'

Необходимую агрегацию по всем фильмам получим с помощью функций map и countByKey

In [24]:
hist_all = ratings.map(lambda x: (x[2], x[2])).countByKey()

Сразу отсортируем список

In [25]:
hist_all = [hist_all[f'{i}'] for i in range(1, 6)]

In [26]:
hist_all

[6110, 11370, 27145, 34174, 21201]

Данные для нашего фильма получим точно так же, но отсортируем dataframe по id

In [50]:
hist_film = ratings.filter(lambda x: x[1] == movie_id).map(lambda x: (x[2], x[3])).countByKey()

In [51]:
hist_film = [hist_film[f'{i}'] for i in range(1, 6)]

In [52]:
hist_film

[12, 40, 94, 109, 40]

In [30]:
answer = {'hist_film' : hist_film, 'hist_all' : hist_all}

In [138]:
import json
with open("lab01.json", "w") as fp:
    json.dump(answer, fp) 


In [53]:
sc.stop()

In [ ]:
! pwd